## import

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install xgboost
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

## input dataset

In [ ]:
df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data_no_multicol_allgender.txt', delimiter='\t')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 1 to 462
Columns: 343 entries, cluster_assignment_sub to A8d_.1.tell_professional
dtypes: float64(160), int64(183)
memory usage: 1.2 MB
None


In [ ]:
df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data_no_multicol_allgender.txt', delimiter='\t')
data=df.iloc[:,1:343]
target=df.iloc[:,0]
target= target.replace(1,0)
target= target.replace(2,1)
target= target.replace(4,2)
target= target.replace(5,3)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 1 to 462
Columns: 343 entries, cluster_assignment_sub to A8d_.1.tell_professional
dtypes: float64(160), int64(183)
memory usage: 1.2 MB
None


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 1 to 462
Columns: 342 entries, CHILDREN to A8d_.1.tell_professional
dtypes: float64(160), int64(182)
memory usage: 1.2 MB
None


## xgboost

In [ ]:
# Initialize an array to store feature importance for each iteration
import numpy as np
feature_importances = np.zeros((50, len(data.columns)))

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107717 sha256=d05c77f40f28147d8d0ac018a677d851c82a1eb1c4a510cce5cc2063d1f430f4
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [ ]:
# repeat 50 times
for i in range(50):
    train_x, test_x, train_y, test_y = train_test_split(data,target,test_size=0.2,random_state=(i*i+5))
    model = XGBClassifier(
        n_estimators=25,
        max_depth=5,
        learning_rate= 0.01,
        subsample=0.75,
        colsample_bytree=0.75,
        reg_lambda=10,
        eval_metric='auc',
        early_stopping_rounds=5,
        random_state=i
    )
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)])
    #perm = PermutationImportance(model, random_state=101).fit(test_x, test_y)
    feature_importances[i, :] = model.feature_importances_

[0]	validation_0-auc:0.89586
[1]	validation_0-auc:0.91557
[2]	validation_0-auc:0.93553
[3]	validation_0-auc:0.93132
[4]	validation_0-auc:0.94846
[5]	validation_0-auc:0.95270
[6]	validation_0-auc:0.95944
[7]	validation_0-auc:0.95849
[8]	validation_0-auc:0.96097
[9]	validation_0-auc:0.96163
[10]	validation_0-auc:0.95954
[11]	validation_0-auc:0.96075
[12]	validation_0-auc:0.96293
[13]	validation_0-auc:0.96291
[14]	validation_0-auc:0.96204
[15]	validation_0-auc:0.96017
[16]	validation_0-auc:0.96394
[17]	validation_0-auc:0.96290
[18]	validation_0-auc:0.96467
[19]	validation_0-auc:0.96647
[20]	validation_0-auc:0.96588
[21]	validation_0-auc:0.96547
[22]	validation_0-auc:0.96543
[23]	validation_0-auc:0.96631
[24]	validation_0-auc:0.96618
[0]	validation_0-auc:0.92086
[1]	validation_0-auc:0.93424
[2]	validation_0-auc:0.92558
[3]	validation_0-auc:0.94439
[4]	validation_0-auc:0.94766
[5]	validation_0-auc:0.95022
[6]	validation_0-auc:0.95149
[7]	validation_0-auc:0.95222
[8]	validation_0-auc:0.94969

## permutation importance

# #result

In [ ]:
# Calculate average feature importance
average_importances = np.mean(feature_importances, axis=0)

# Sort and get the index of the top 5 features
top_10_feature_indices = np.argsort(average_importances)[-10:]

# Get and print the top 5 most important feature names and their average importance
top_features = data.columns[top_10_feature_indices]
for feature, importance in zip(top_features, average_importances[top_10_feature_indices]):
    print(f"Feature: {feature}, Importance: {importance}")

Feature: GAD_2, Importance: 0.014308883438352495
Feature: Smokebefore, Importance: 0.015220294687896967
Feature: Qb.height_inches, Importance: 0.01522722298745066
Feature: PHQ_4, Importance: 0.016826497223228216
Feature: PHQ_7, Importance: 0.018189741373062132
Feature: Q4_Currently_Smoke, Importance: 0.018720952086150645
Feature: GAD_3, Importance: 0.02861436895094812
Feature: Qa_height_feet, Importance: 0.03633729230612517
Feature: GAD_4, Importance: 0.047874419912695886
Feature: gender, Importance: 0.08105445258319378
